In [2]:
import sys

sys.path.append('/Users/valuamba/projs/components_agent_sales/app/utils')
sys.path.append('/Users/valuamba/projs/components_agent_sales/app/core/clients/')

from html_messages_parser import get_element_messages, get_messages_from_html_file, select_json_block 
from famaga import FamagaClient

In [3]:
# let's define a few helper utilities

from pathlib import Path
import hashlib
import subprocess
import json
from bs4 import BeautifulSoup


text_cache = Path('cache')

def sha1(input_string):
    """Helper to hash input strings"""
    try:

        # Step 5: Create a new SHA-1 hash object
        hash_object = hashlib.sha1()

        # Step 6: Update the hash object with the bytes-like object
        hash_object.update(input_string.encode('utf-8'))

        # Step 7: Get the hexadecimal representation of the hash
        return hash_object.hexdigest()
    except Exception as e:
        raise ValueError(input_string) from e
        
def htmltotext(source: str):
    """Extract text from Email. Requires pdftotext binary in the path"""

    with open(source, 'r') as f:
        data = f.read()

    soup = BeautifulSoup(data, 'html.parser')
    
    return soup.get_text()

def extract_text(source):
    """Get text from Email. Cache results to avoid recomputation"""
    local = text_cache / sha1(source.name + " > text")

    if local.exists():
        return local
    print(f"extracting from {source}")
    print(f'target: {local}')
    htmltotext(source, local)
    return local


def email_to_text(path):
    return extract_text(path).read_text()

In [4]:
from functools import wraps
import inspect


def stored(func):
    """
    implements nix-like durable memoisation of function results.

    Lazy way to avoid recomputing expensive calls. Expects results to be JSON-serializable
    """
    @wraps(func)
    def CACHE(*args, **kwargs):
        name = func.__name__
        meta = {}

        meta["name"] = name
        meta["func"] = inspect.getsource(func)
        meta["args"] = args
        meta["kwargs"] = kwargs

        js = json.dumps(meta)
        sha = hashlib.sha1(js.encode('utf-8'))

        digest = sha.hexdigest()

        path = text_cache / f"{digest}-{name}.json"

        if path.exists():
            with path.open('r') as r:
                cached = json.load(r)
            return cached["result"]
        result = func(*args, **kwargs)
        meta["result"] = result
        with path.open('w') as w:
            json.dump(meta, w)
        return result

    return CACHE

In [5]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

@stored
def get_gpt(content, model="gpt-4-1106-preview", temperature=0, max_tokens=1000, stream=True):
    """
    Cached call to GPT.
    """
    messages = [{"role": "user", "content": content}]

    if stream:
        response = client.chat.completions.create(
            model=model, 
            messages=messages, 
            temperature=temperature, 
            stream=True
        )
        
        collected_messages = []
        for chunk in response:
            if chunk.choices[0].delta.content:
                print(chunk.choices[0].delta.content, end='')
                collected_messages.append(chunk.choices[0].delta.content)
    
        content_str = ''.join(collected_messages)
        return content_str
    else:
        completion = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return completion.model_dump()


In [14]:
get_gpt('What is the lol kek cheburek?1', 'gpt-4')

"Lol Kek Cheburek" is a phrase that originated from Russian internet slang. "Lol" is an English internet slang term meaning "laugh out loud", "Kek" is a Korean internet slang term used to express laughter, similar to "lol", and "Cheburek" is a popular deep-fried turnover with a filling of ground or minced meat and onions. It is often used in memes or as a humorous phrase online.

'"Lol Kek Cheburek" is a phrase that originated from Russian internet slang. "Lol" is an English internet slang term meaning "laugh out loud", "Kek" is a Korean internet slang term used to express laughter, similar to "lol", and "Cheburek" is a popular deep-fried turnover with a filling of ground or minced meat and onions. It is often used in memes or as a humorous phrase online.'

In [ ]:
7f2eee719c076609a1f11f770bf9513f1c1f94e0-get_gpt

In [13]:
name1 = '502048_78717.0.html'
name = '467721_51407.0.html'

deal_id = name.split('_')[0]

text = htmltotext(Path(f'./deals_html/discount_v4/{name}'))

In [31]:
@stored
def get_messages(txt):

    prompt = """
You are Sales manager. Read a conversation between customer and manager and classify messages, senders (customer or manager).

Respond with array of messagds. Respond with an empty string, if none is found

```txt
$TXT
```""".strip() + "\n"
    
    filled = prompt.replace("$TXT", txt)
    result = get_gpt(filled)
    return result

In [35]:
result = get_messages(text)
result

'```json\n[\n  {\n    "message": "Please give the price in words for better understanding. also mention HSCode",\n    "sender": "customer"\n  },\n  {\n    "message": "We highly appreciate your offer, but prices are much higher than our budgeted figures. Hence request to offer special discount enabling us to get approval from our management. Delivery period should be shortened as much as possible.",\n    "sender": "customer"\n  },\n  {\n    "message": "Hello Mr Shumail, Supplier have offered for base please find attached the TDS and select the complete code. Can you please check the item number again? Unfortunately this is not complete.I have attached the Schaltbau catalog for you. You will find the order key on page 4. Thank you Best Regards, Pushkar Agarwal +97156.9576861",\n    "sender": "manager"\n  },\n  {\n    "message": "Offer-Nr.:502048 Customer request #:Shumail Customer #:95613 Date:Tuesday, 23 April 2024 Inquiry #:Contact person:Pushkar Agarwal Offer valid till22.05.2024 Inqu

In [38]:
@stored
def discount_chronology(txt):

    prompt = """
You are Sales manager. Read a conversation between customer and manager and classify discount chronology discussion.

Respond with array of messages where was mentioned or discussed discount.
Respond with an empty string, if none is found

```txt
$TXT
```""".strip() + "\n"
    
    filled = prompt.replace("$TXT", txt)
    result = get_gpt(filled)
    return result

In [44]:
messages = []
for msg in select_json_block(result):
    if 'Offer-Nr.' not in msg['message']:
        messages.append(msg)
    else:
        messages.append({ 'message': '< Commercial offer >', 'sender': 'maanger'})


chronology = discount_chronology('\n'.join([f'Message: {m['message']}\nSender: {m['sender']}' for m in messages]))     

```json
[
  "We highly appreciate your offer, but prices are much higher than our budgeted figures. Hence request to offer special discount enabling us to get approval from our management. Delivery period should be shortened as much as possible."
]
```

In [49]:
api_key = "YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw"
session_id = "085qpt4eflu39a0dg7hjhr5mdu"
famaga = FamagaClient(api_key, session_id)

def get_customer_data(deal_id):
    offer_info = famaga.list_offers_by_deal_id(deal_id)
    offer_id = offer_info['content'][0]['request']['id']
    client_id = offer_info['content'][0]['request']['firm']['id']
    
    purchase_history = famaga.get_client_purchase_history_formatted(client_id, int(deal_id))
    
    current_offer = famaga.list_current_offer_details(offer_id)

    return purchase_history, current_offer

In [50]:
get_customer_data(502048)

('[CLIENT PURCHASE HISTORY]\n**Client purchase history:**\nDate: 2022-09-29\n272381 (F862BP224K310Z Kemet/Evox Rifa) margin: 85%, sell: 22.47$ qty. 10, request id 350875\n\nDate: 2024-04-23\n375707 (T77630-40 AI-TEK Instruments) margin: 22%, sell: 3315.87$ qty. 1, request id 502049\n\nDate: 2024-04-25\n376251 (665006002 Mystik) margin: 25%, sell: 906.38$ qty. 1, request id 503139\n\n[/CLIENT PURCHASE HISTORY]',
 '(S800E Schaltbau) margin: 25%, sell: 101.59$ qty. 6')

In [15]:
import requests
import uuid
import json

data = {
    "run_uuid": str(uuid.uuid4()),
    "deal_id": deal_id,
    "raw_text": text
}


response = requests.post('http://localhost:8013/v2/agent/discount/raw_text', data=json.dumps(data))

response.content

b'{"actions":[{"action":{"action_version":1,"action_name":"classify_discount_messages","action_id":6627,"action_time":10904.0,"action_status":1},"ui_message":"<b>Action 6627 - classify_discount_messages</b> \xf0\x9f\x9f\xa2 Passed\\n\\nMessages amount: 3\\n","data":[{"message":"Dear Sir,\xc2\xa0Please provide your official Quotation along with shorten delivery lead time.Also please provide us discounted prices to issue the PO.\xc2\xa0Thanks & Best Regards,\xc2\xa0\xc2\xa0\xc2\xa0Sameer Iqbal","sender":"customer"},{"message":"*\xc2\xa0Though the supplier has increased the price by 5% in 2024 we have reduced our margin and offered you the same.","sender":"manager"},{"message":"Dear Sir,\xc2\xa0Please provide your best discounted prices for below BOQ\xe2\x80\x99s, to get our deep look on your Quote.Also provide us Weight & Dimension of Shipment.","sender":"customer"}],"metadata":{"completion_cost_usd":0.059,"completion_time_sec":10904.0,"llm_model":"gpt-4","raw_output":"```json\\n[\\n    

In [16]:
print(json.dumps(response.json(), indent=2))

{
  "actions": [
    {
      "action": {
        "action_version": 1,
        "action_name": "classify_discount_messages",
        "action_id": 6627,
        "action_time": 10904.0,
        "action_status": 1
      },
      "ui_message": "<b>Action 6627 - classify_discount_messages</b> \ud83d\udfe2 Passed\n\nMessages amount: 3\n",
      "data": [
        {
          "message": "Dear Sir,\u00a0Please provide your official Quotation along with shorten delivery lead time.Also please provide us discounted prices to issue the PO.\u00a0Thanks & Best Regards,\u00a0\u00a0\u00a0Sameer Iqbal",
          "sender": "customer"
        },
        {
          "message": "*\u00a0Though the supplier has increased the price by 5% in 2024 we have reduced our margin and offered you the same.",
          "sender": "manager"
        },
        {
          "message": "Dear Sir,\u00a0Please provide your best discounted prices for below BOQ\u2019s, to get our deep look on your Quote.Also provide us Weight & Di